# not working:

In [ ]:
system_template = """<Context>You are a helpful assistant that translates {language} comments (short texts) to English.</Context>

<Data Structure>The lists "translate" is an array containing {language} short texts.</Data Structure>

<Task>Translate the provided array "translate" to English, whereby you return a dictionary with the {language} comments as keys and their English translations as values. 
The dictionary should be structured according to the provided JSON schema.
Be careful with spelling errors and do not use capital letters for adjectives.
Repeat this process in case no translations are returned or the dictionary do not follow the JSON schema.
Check if you translated the complete comment.</Task>
"""

user_template = """Array "translate": 
{translate}
"""

# rescue robots and socially assistive robots
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", user_template)]
)


result = prompt_template.invoke({"language": "German", "translate":["text1", "text2"]})
print(result)

print("result:", result)
print("result.to_messages():", result.to_messages())

In [ ]:
json_schema = {
    "title": "Translation",
    "description": "Dictionary that contains the comments and their respective translations.",
    "type": "object",
    "properties": {
        "dictionary": {
            "type": "string",
            "description": 'Dictionary that contains the words as keys and their respective translations as values. Provide the dictionary in the following format: {"comment1": "translation1", "comment2": "translation2", ...}.',
        },
    },
    "required": ["dictionary"],
}

In [ ]:
save_interval = 10

dictionary_out = {}
counter = 0
# Loop through the list in chunks
chunk_size = 5
for i in range(0, len(df_comment), chunk_size):
    # Select the next XX words (or fewer, if there are less than XX remaining)
    print(i)
    counter += 1
    chunk = df_comment["comment"][i:i + chunk_size]
    ids = df_comment["id"][i:i + chunk_size]
    
    print(len(chunk))
    print(len(ids))

    result = basic_API_call(prompt=prompt_template,
    language="German",
    translate=chunk,
    openai_api_key=key.openai_api_key,
    json_schema=json_schema,
    model_name="gpt-4o",
    max_tokens=1000)
    
    # Extract the 'parsed' section from the JSON data
    parsed_section = result.get('parsed', {})
    dictionary = parsed_section.get('dictionary')
    
    # Convert the string back to a Python dictionary
    #print(type(dictionary)) 
    dictionary = eval(dictionary)

    #print(len(dictionary.keys()))
    #print(dictionary) 

    if i == 0:
        dictionary_out = dictionary
    else:
        #break
        dictionary_out.update(dictionary)
       
    # create a DF every time to check if no words were missed 
    df_temp = pd.DataFrame(list(dictionary.items()), columns=['German', 'English'])
    df_temp['ID']= df_comment["id"][i:i + chunk_size].reset_index(drop=True)
    
    if i == 0:
        df_out = df_temp
    else:
        df_out = pd.concat([df_out, df_temp], axis=0)

    if counter % save_interval == 0:
        df_out.to_excel(f'translations_comments_up_to_{counter}.xlsx', index=False)
        
        

# Save the DataFrame to an Excel file
df_out.to_excel('translations_comments.xlsx', index=False)
print("Translations saved to translations_comments.xlsx")


In [ ]:
# Step 1: Set up the OpenAI API
client = OpenAI(
    api_key=key.openai_api_key,
)

# Define the array of German words
words_to_translate = df_text


# Function to translate a word using ChatGPT API
def translate_word(word):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that translates German words to English."},
                {"role": "user", "content": f'Translate the following German word to English: "{word}" Be careful with spelling errors and do not use capital letters for adjectives. Only return the translation in quotes.'}
            ]
        )
        # print(response.choices[0].message.content.strip())
        
        #content = response.choices[0].message.content.strip()
        #start_index = content.find('is "') + 4
        #end_index = content.find('"', start_index)
        #translation = content[start_index:end_index]
        #print(translation)
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error translating word '{word}': {e}")
        return None

# Step 2: Translate the words
translations = [translate_word(word) for word in words_to_translate]

# Remove None values if any translation fails
translated_words = [word for word, translation in zip(words_to_translate, translations) if translation is not None]
translations = [translation for translation in translations if translation is not None]

# Step 3: Store the translations in a .xlsx file
df_translated_text = pd.DataFrame({
    'German': translated_words,
    'English': translations
})
print(df_translated_text)

In [ ]:
# Step 1: Set up the OpenAI API
client = OpenAI(
    api_key=key.openai_api_key,
)

# Define the array of German words
comments_to_translate = df_comment['comment']
comments_ids_to_translate = df_comment['id']

# Function to translate a word using ChatGPT API
def translate_comment(comment):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that translates German comments to English."},
                {"role": "user", "content": f'Translate the following German comment to English: "{comment}" Be careful with spelling errors and do not use capital letters for adjectives. Only return the translation in quotes.'}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error translating comment '{comment}': {e}")
        return None

# Step 2: Translate the words with logging and saving
translations = []
counter = 0
save_interval = 50
print_interval = 100

for comment in comments_to_translate:
    translation = translate_comment(comment)
    if translation:
        translations.append(translation)
    counter += 1
    if counter % print_interval == 0:
        print(f"Current counter: {counter}")
    if counter % save_interval == 0:
        df_temp = pd.DataFrame({'ID': comments_ids_to_translate[:counter], 'German': comments_to_translate[:counter], 'English': translations})
        df_temp.to_excel(f'translations_comments_up_to_{counter}.xlsx', index=False)

# Final save
df_translated_comments = pd.DataFrame({
    'ID': comments_ids_to_translate,
    'German': comments_to_translate,
    'English': translations
})
df_translated_comments.to_excel('translations_comments.xlsx', index=False)

print(df_translated_comments)
